#Zadanie 7 (7 pkt)
Celem zadania jest zaimplementowanie dwóch wersji naiwnego klasyfikatora Bayesa.
* W pierwszej wersji należy dokonać dyskretyzacji danych - przedział wartości każdego atrybutu dzielimy na cztery równe przedziały i każdej ciągłej wartości atrybutu przypisujemy wartość dyskretną wynikająca z przynależności do danego przedziału.
* W drugiej wersji wartości likelihood wyliczamy z rozkładów normalnych o średnich i odchyleniach standardowych wynikających z wartości atrybutów.
Trening i test należy przeprowadzić dla zbioru Iris, tak jak w przypadku zadania z drzewem klasyfikacyjnym. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania klasyfikatorów dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Dyskretyzacja danych - **0.5 pkt**
* Implementacja funkcji rozkładu normalnego o zadanej średniej i odchyleniu standardowym. - **0.5 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych dyskretnych. - **2.0 pkt**
* Implementacja naiwnego klasyfikatora Bayesa dla danych ciągłych. - **2.5 pkt**
* Przeprowadzenie eksperymentów, wnioski i sposób ich prezentacji. - **1.5 pkt**

In [134]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [135]:
y_test == 1

array([ True, False, False,  True, False, False,  True, False, False,
        True, False, False,  True, False, False])

Dyskretyzacja lewostronnie domknięta

In [136]:
class NaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        discrete_xtrain = self.data_discretization(train_features)
        labels = np.unique(train_classes)
        labels_count = Counter(train_classes)

        for label in labels:
            self.priors[label] = labels_count[label] / len(train_classes)

            label_dict = {}
            mask = train_classes == label
            for i in range(discrete_xtrain.shape[1]):
                attribute = discrete_xtrain[mask, i]
                attribute_counter = Counter(attribute)
                attribute_dict = {x: 0 for x in np.unique(discrete_xtrain)}
                for unique_value, number in attribute_counter.items():
                    print(number, labels_count[label])
                    attribute_dict[unique_value] = number / labels_count[label]

                label_dict[i] = attribute_dict
            self.likelihoods[label] = label_dict

    @staticmethod
    def data_discretization(data: np.ndarray) -> np.ndarray:
        intervals = 4

        discrete_array = np.zeros(data.shape)
        for i in range(data.shape[1]):
            min_value = np.min(data[:, i])
            max_value = np.max(data[:, i])
            interval = (max_value - min_value) / intervals
            for j in range(data.shape[0]):
                discrete_array[j, i] = min((data[j, i] - min_value) // interval,
                                           intervals - 1)

        return discrete_array

    def predict(self, sample):
        predictions = {}
        for label, prior in self.priors.items():
            value = 1
            for i, att_value in enumerate(sample):
                value *= self.likelihoods[label][i][att_value]
            predictions[label] = value

        return max(predictions, key=predictions.get)


class GaussianNaiveBayes:
    def __init__(self):
        self.priors = {}
        self.likelihoods = {}

    def build_classifier(self, train_features, train_classes):
        labels = np.unique(train_classes)
        labels_count = Counter(train_classes)

        for label in labels:
            self.priors[label] = labels_count[label] / len(train_classes)

            label_dict = {}
            mask = train_classes == label
            for i in range(train_features.shape[1]):
                attribute = train_features[mask, i]
                attribute_dict = {x: 0 for x in np.unique(train_features)}

                attribute_mean = np.mean(attribute)
                attribute_std = np.std(attribute)

                for value in np.unique(attribute):
                    attribute_dict[value] = self.normal_dist(value,
                                                             attribute_mean,
                                                             attribute_std)

                label_dict[i] = attribute_dict
            self.likelihoods[label] = label_dict

    @staticmethod
    def normal_dist(x, mean, std):
        prob_density = (np.pi * std) * np.exp(-0.5 * ((x - mean) / std) ** 2)
        return prob_density

    def predict(self, sample):
        predictions = {}
        for label, prior in self.priors.items():
            value = 1
            for i, att_value in enumerate(sample):
                value *= self.likelihoods[label][i][att_value]
            predictions[label] = value

        return max(predictions, key=predictions.get)

In [137]:
bayes = NaiveBayes()

In [138]:
bayes.build_classifier(x_train, y_train)

13 46
33 46
30 46
10 46
5 46
1 46
46 46
46 46
27 45
4 45
14 45
29 45
11 45
5 45
35 45
10 45
31 45
14 45
23 44
9 44
11 44
1 44
26 44
13 44
5 44
24 44
20 44
25 44
19 44


In [139]:
bayes.priors

{0: 0.34074074074074073, 1: 0.3333333333333333, 2: 0.32592592592592595}

In [140]:
bayes.likelihoods

{0: {0: {0.0: 0.717391304347826, 1.0: 0.2826086956521739, 2.0: 0, 3.0: 0},
  1: {0.0: 0.021739130434782608,
   1.0: 0.21739130434782608,
   2.0: 0.6521739130434783,
   3.0: 0.10869565217391304},
  2: {0.0: 1.0, 1.0: 0, 2.0: 0, 3.0: 0},
  3: {0.0: 1.0, 1.0: 0, 2.0: 0, 3.0: 0}},
 1: {0: {0.0: 0.08888888888888889, 1.0: 0.6, 2.0: 0.3111111111111111, 3.0: 0},
  1: {0.0: 0.24444444444444444,
   1.0: 0.6444444444444445,
   2.0: 0.1111111111111111,
   3.0: 0},
  2: {0.0: 0, 1.0: 0.2222222222222222, 2.0: 0.7777777777777778, 3.0: 0},
  3: {0.0: 0, 1.0: 0.3111111111111111, 2.0: 0.6888888888888889, 3.0: 0}},
 2: {0: {0.0: 0.022727272727272728,
   1.0: 0.20454545454545456,
   2.0: 0.5227272727272727,
   3.0: 0.25},
  1: {0.0: 0.11363636363636363,
   1.0: 0.5909090909090909,
   2.0: 0.29545454545454547,
   3.0: 0},
  2: {0.0: 0, 1.0: 0, 2.0: 0.45454545454545453, 3.0: 0.5454545454545454},
  3: {0.0: 0, 1.0: 0, 2.0: 0.4318181818181818, 3.0: 0.5681818181818182}}}

In [141]:
x_test = bayes.data_discretization(x_test)
positive = 0
for data, label in zip(x_test, y_test):
    pred = bayes.predict(data)
    print(pred, label)
    if pred == label:
        positive += 1

print(f"Accuracy: {(positive/len(y_test))*100:.2f}%")

2 1
2 2
2 2
1 1
0 0
2 2
1 1
0 0
0 0
1 1
2 2
0 0
1 1
2 2
2 2
Accuracy: 93.33%
